### Load the dataset

In [1]:
import pandas as pd
dataset = pd.read_csv('datasets/covid19-twitter.tsv', sep="\t")
dataset

,tweet_id,emojis,emoji_names,emotion_scores,agg_emotion_score,sentiment_label,vader_score_sentence,vader_score_clause_A,vader_score_clause_B,vader_sentiment_sentence,vader_sentiment_clause_A,vader_sentiment_clause_B,rule_structure,rule_label,contrast
0,1292641373758328839,"['😢', '😢']","['crying face', 'crying face']","[[-0.25, -0.36, -0.5, 0.0, -1.0, 0.11], [-0.25...",-4.00,negative,"{'neg': 0.249, 'neu': 0.751, 'pos': 0.0, 'comp...",not_applicable,not_applicable,negative,not_applicable,not_applicable,no_structure,not_applicable,not_applicable
1,1377497788460720129,"['😂', '😂', '😂', '😂']","['face with tears of joy', 'face with tears of...","[[0.0, -0.06, -0.06, 0.94, 0.0, 0.22], [0.0, -...",4.16,positive,"{'neg': 0.021, 'neu': 0.909, 'pos': 0.07, 'com...","{'neg': 0.036, 'neu': 0.909, 'pos': 0.056, 'co...","{'neg': 0.0, 'neu': 0.9, 'pos': 0.1, 'compound...",positive,positive,positive,A-but-B,A-but-B,no_contrast
2,1318434591322087427,"['💔', '💔', '💔']","['broken heart', 'broken heart', 'broken heart']","[[-0.39, -0.33, -0.14, 0.0, -0.94, 0.17], [-0....",-4.89,negative,"{'neg': 0.176, 'neu': 0.824, 'pos': 0.0, 'comp...",not_applicable,not_applicable,negative,not_applicable,not_applicable,no_structure,not_applicable,not_applicable
3,1303627865796222976,"['😠', '😠', '😠', '😠', '😠']","['angry face', 'angry face', 'angry face', 'an...","[[-1.0, -0.56, -0.17, 0.0, -0.25, 0.08], [-1.0...",-9.50,negative,"{'neg': 0.117, 'neu': 0.883, 'pos': 0.0, 'comp...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...","{'neg': 0.302, 'neu': 0.698, 'pos': 0.0, 'comp...",negative,neutral,negative,A-yet-B,A-yet-B,contrast
4,1361789228426932227,"['🎉', '🎉', '🎉']","['party popper', 'party popper', 'party popper']","[[0.0, 0.0, 0.0, 0.92, 0.0, 0.33], [0.0, 0.0, ...",3.75,positive,"{'neg': 0.0, 'neu': 0.686, 'pos': 0.314, 'comp...",not_applicable,not_applicable,positive,not_applicable,not_applicable,no_structure,not_applicable,not_applicable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109914,1304993476564590592,"['😡', '😡', '😡']","['pouting face', 'pouting face', 'pouting face']","[[-1.0, -0.56, -0.11, 0.0, -0.36, 0.06], [-1.0...",-5.91,negative,"{'neg': 0.192, 'neu': 0.808, 'pos': 0.0, 'comp...",not_applicable,not_applicable,negative,not_applicable,not_applicable,no_structure,not_applicable,not_applicable
109915,1389249469850394627,"['💐', '💐', '💐', '💐', '👍']","['bouquet', 'bouquet', 'bouquet', 'bouquet', '...","[[0.0, 0.0, 0.0, 0.69, -0.11, 0.58], [0.0, 0.0...",5.19,positive,"{'neg': 0.0, 'neu': 0.291, 'pos': 0.709, 'comp...",not_applicable,not_applicable,positive,not_applicable,not_applicable,no_structure,not_applicable,not_applicable
109916,1322678255128903683,"['😢', '😢']","['crying face', 'crying face']","[[-0.25, -0.36, -0.5, 0.0, -1.0, 0.11], [-0.25...",-4.00,negative,"{'neg': 0.106, 'neu': 0.846, 'pos': 0.049, 'co...","{'neg': 0.0, 'neu': 0.822, 'pos': 0.178, 'comp...","{'neg': 0.133, 'neu': 0.867, 'pos': 0.0, 'comp...",negative,positive,negative,A-but-B,A-but-B,contrast
109917,1361083777724915712,"['🙏', '🙏', '🙏', '🙏', '🙏', '🙏', '🙏', '🙏', '🙏', ...","['folded hands', 'folded hands', 'folded hands...","[[-0.06, 0.0, -0.11, 0.25, -0.11, 0.58], [-0.0...",5.50,positive,"{'neg': 0.0, 'neu': 0.79, 'pos': 0.21, 'compou...","{'neg': 0.0, 'neu': 0.748, 'pos': 0.252, 'comp...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",positive,positive,neutral,A-while-B,A-while-B,contrast


### Hydrate tweets from tweet-IDs using tweepy

In [ ]:
import tweepy

bearer_token="YOUR_BEARER_TOKEN_VALUE"
api_key = 'YOUR_API_KEY_VALUE'
api_secret = 'YOUR_API_SECRET_VALUE'
access_token = 'YOUR_ACCESS_TOKEN_VALUE'
access_token_secret = 'YOUR_ACCESS_TOKEN_SECRET_VALUE'

client = tweepy.Client(bearer_token)
client = tweepy.Client(consumer_key=consumer_key, 
                       consumer_secret=consumer_secret,
                       access_token=access_token, 
                       access_token_secret=access_token_secret)

tweet_ids = list(dataset["tweet_id"])
response = client.get_tweets(tweet_ids)

tweets = []
for index, tweet in enumerate(response.data):
    assert(tweet.id==tweet_ids[index])
    tweets.append(tweet.text)
dataset["tweet"] = tweets

In [1]:
#Install packages
!pip install pandas
!pip install tqdm
!pip install tweet-preprocessor
!pip install scikit-learn
!pip install lime
!pip install keras-tuner
!pip install transformers

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/fc/a5/4d82be566f069d7a9a702dcdf6f9106df0e0b042e738043c0cc7ddd7e3f6/pandas-2.2.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/9c/3d/a121f284241f08268b21359bd425f7d4825cffc5ac5cd0e1b3d82ffd2b10/pytz-2024.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for tzdata>=2022.7 from https://files.pythonhosted.org/packages/65/58/f9c9e6be752e9fcb8b6a0ee9fb87e6e7a1f6bcab2cdc73f02bb7ba91ada0/tzdata-2024.1-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 27.0 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 10.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 8.1 MB/s eta 0:00:00:00:01

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: 

### Reproduce benchmark results

In [2]:
#specifiy the parameters
config = {
            "asset_name":"bertweet_lstm_reproducibility_test",
            "model_name":"bertweet_lstm",
            "seed_value":11,
            "dataset_name":"covid19-twitter",
            "train_model":True,
            "evaluate_model":True,
            "generate_explanations":True,
            "generate_explanation_for_one_instance":True,
            "fine_tune_word_embeddings":True, # for cwe classifiers: True, for flat classifiers: False
            "optimizer":"adam",
            "learning_rate":1e-6,
            "mini_batch_size":50,
            "train_epochs":2,
            "dropout":0.5,
            "lime_no_of_samples":1000,
            "hidden_units":128  #for mlp, lstm, bilstm, gru, bigru: 128, for cnn: 100 (no of filters), for transformer: 2 (no of heads)
        }

In [3]:
#import packages
import argparse
import pprint
import os
import logging
import warnings
import random
import numpy as np
import pickle
import pandas as pd
import tensorflow as tf
print("\nNum GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
import argparse
import subprocess as sp
import distutils
import pprint

#change the code execution directory to current directory
os.chdir(os.getcwd())

#scripts
from scripts.preprocess_dataset import Preprocess_dataset
from scripts.word_vectors import Word_vectors
from scripts.dataset_division import Dataset_division

from scripts.train_mlp import train_mlp
from scripts.train_transformer import train_transformer
from scripts.train_cnn import train_cnn
from scripts.train_rnn import train_rnn

from scripts.train_bertweet_mlp import train_bertweet_mlp
from scripts.train_bertweet_transformer import train_bertweet_transformer
from scripts.train_bertweet_cnn import train_bertweet_cnn
from scripts.train_bertweet_rnn import train_bertweet_rnn
# from scripts.train_gpt2_mlp import train_gpt2_mlp
# from scripts.train_gpt2_transformer import train_gpt2_transformer
# from scripts.train_gpt2_rnn import train_gpt2_rnn
# from scripts.train_gpt2_cnn import train_gpt2_cnn
# from scripts.train_roberta_mlp import train_roberta_mlp
# from scripts.train_roberta_cnn import train_roberta_cnn
# from scripts.train_roberta_transformer import train_roberta_transformer
# from scripts.train_roberta_rnn import train_roberta_rnn
# from scripts.train_xlnet_mlp import train_xlnet_mlp
# from scripts.train_xlnet_cnn import train_xlnet_cnn
# from scripts.train_xlnet_transformer import train_xlnet_transformer
# from scripts.train_xlnet_rnn import train_xlnet_rnn
# from scripts.train_distilbert_mlp import train_distilbert_mlp
# from scripts.train_distilbert_cnn import train_distilbert_cnn
# from scripts.train_distilbert_transformer import train_distilbert_transformer
# from scripts.train_distilbert_rnn import train_distilbert_rnn
# from scripts.train_elmo_mlp import train_elmo_mlp

#disable warnings
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
warnings.filterwarnings("ignore")

#set the gpu execution device with highest free memory
def mask_unused_gpus(leave_unmasked=1): # No of avaialbe GPUs on the system
    COMMAND = "nvidia-smi --query-gpu=memory.free --format=csv"
    try:
        _output_to_list = lambda x: x.decode('ascii').split('\n')[:-1]
        memory_free_info = _output_to_list(sp.check_output(COMMAND.split()))[1:]
        memory_free_values = [int(x.split()[0]) for i, x in enumerate(memory_free_info)]
        available_gpus = [i for i, x in enumerate(memory_free_values)]
        if len(available_gpus) < leave_unmasked: raise ValueError('Found only %d usable GPUs in the system' % len(available_gpus))
        gpu_with_highest_free_memory = 0
        highest_free_memory = 0
        for index, memory in enumerate(memory_free_values):
            if memory > highest_free_memory:
                highest_free_memory = memory
                gpu_with_highest_free_memory = index
        return str(gpu_with_highest_free_memory)
    except Exception as e:
        print('"nvidia-smi" is probably not installed. GPUs are not masked', e)
os.environ["CUDA_VISIBLE_DEVICES"] = mask_unused_gpus()

#prepare the dataset
print("\nCreating input data")
preprocessed_dataset = Preprocess_dataset(config).preprocess_covid_tweets(dataset)
word_vectors, word_index = Word_vectors(config).create_word_vectors(preprocessed_dataset)
train_dataset, val_datasets, test_datasets = Dataset_division(config).train_val_test_split(preprocessed_dataset)

#create model
print("\nBuilding model")

#flat classifiers
if config["model_name"] == "mlp":
    train_mlp(config).train_model(train_dataset, val_datasets, test_datasets, word_index, word_vectors)
elif config["model_name"] == "transformer":
    train_transformer(config).train_model(train_dataset, val_datasets, test_datasets, word_index, word_vectors)
elif config["model_name"] == "cnn":
    train_cnn(config).train_model(train_dataset, val_datasets, test_datasets, word_index, word_vectors)
elif config["model_name"] in ["lstm", "bilstm", "gru", "bigru"]:
    train_rnn(config).train_model(train_dataset, val_datasets, test_datasets, word_index, word_vectors)

#cwe classifiers
elif config["model_name"] == "bertweet_mlp":
    train_bertweet_mlp(config).train_model(train_dataset, val_datasets, test_datasets)
elif config["model_name"] == "bertweet_transformer":
    train_bertweet_transformer(config).train_model(train_dataset, val_datasets, test_datasets)
elif config["model_name"] == "bertweet_cnn":
    train_bertweet_cnn(config).train_model(train_dataset, val_datasets, test_datasets)
elif config["model_name"] in ["bertweet_lstm", "bertweet_bilstm", "bertweet_gru", "bertweet_bigru"]:
    train_bertweet_rnn(config).train_model(train_dataset, val_datasets, test_datasets)
elif config["model_name"] == "gpt2_mlp":
    train_gpt2_mlp(config).train_model(train_dataset, val_datasets, test_datasets)
elif config["model_name"] == "gpt2_transformer":
    train_gpt2_transformer(config).train_model(train_dataset, val_datasets, test_datasets)
elif config["model_name"] in ["gpt2_lstm", "gpt2_bilstm", "gpt2_gru", "gpt2_bigru"]:
    train_gpt2_rnn(config).train_model(train_dataset, val_datasets, test_datasets)
elif config["model_name"] == "gpt2_cnn":
    train_gpt2_cnn(config).train_model(train_dataset, val_datasets, test_datasets)
elif config["model_name"] == "roberta_mlp":
    train_roberta_mlp(config).train_model(train_dataset, val_datasets, test_datasets)
elif config["model_name"] == "roberta_cnn":
    train_roberta_cnn(config).train_model(train_dataset, val_datasets, test_datasets)
elif config["model_name"] == "roberta_transformer":
    train_roberta_transformer(config).train_model(train_dataset, val_datasets, test_datasets)
elif config["model_name"] in ["roberta_lstm", "roberta_bilstm", "roberta_gru", "roberta_bigru"]:
    train_roberta_rnn(config).train_model(train_dataset, val_datasets, test_datasets)
elif config["model_name"] == "xlnet_mlp":
    train_xlnet_mlp(config).train_model(train_dataset, val_datasets, test_datasets)
elif config["model_name"] == "xlnet_cnn":
    train_xlnet_cnn(config).train_model(train_dataset, val_datasets, test_datasets)
elif config["model_name"] == "xlnet_transformer":
    train_xlnet_transformer(config).train_model(train_dataset, val_datasets, test_datasets)
elif config["model_name"] in ["xlnet_lstm", "xlnet_bilstm", "xlnet_gru", "xlnet_bigru"]:
    train_xlnet_rnn(config).train_model(train_dataset, val_datasets, test_datasets)
elif config["model_name"] == "distilbert_mlp":
    train_distilbert_mlp(config).train_model(train_dataset, val_datasets, test_datasets)
elif config["model_name"] == "distilbert_cnn":
    train_distilbert_cnn(config).train_model(train_dataset, val_datasets, test_datasets)
elif config["model_name"] == "distilbert_transformer":
    train_distilbert_transformer(config).train_model(train_dataset, val_datasets, test_datasets)
elif config["model_name"] in ["distilbert_lstm", "distilbert_bilstm", "distilbert_gru", "distilbert_bigru"]:
    train_distilbert_rnn(config).train_model(train_dataset, val_datasets, test_datasets)

2024-06-11 13:56:02.039850: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-11 13:56:02.039889: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-11 13:56:02.039915: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-11 13:56:02.047803: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.



Num GPUs Available:  1

Creating input data


2024-06-11 13:57:03.266389: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9689 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5



Word vectors created

Converted 36063 words (27654 misses)

Building model
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 81)]                 0         []                            
                                                                                                  
 tf.cast (TFOpLambda)        (None, 81)                   0         ['input_1[0][0]']             
                                                                                                  
 tf.math.not_equal (TFOpLam  (None, 81)                   0         ['tf.cast[0][0]']             
 bda)                                                                                             
                                                                                                  
 tf.cast_1 (TFOpLa

2024-06-11 13:58:39.929288: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600


1407/1407 [==============================] - 725s 504ms/step - loss: 0.5685 - accuracy: 0.7064 - val_loss: 0.4643 - val_accuracy: 0.7947
Epoch 2/2
687/687 [==============================] - 65s 95ms/step - loss: 0.3632 - accuracy: 0.8482
test loss, test acc: [0.36320289969444275, 0.8482077717781067]
687/687 [==============================] - 45s 63ms/step
21984


0.876

LIME explanations


  0%|                                                                                                                                                                                                                                                                                               | 0/10677 [00:00<?, ?it/s]

1/1 [==============================] - 5s 5s/step


  0%|                                                                                                                                                                                                                                                                                               | 0/10677 [00:07<?, ?it/s]

[0.004445814143050769, 0.0028624276012147003, 0.0016117015342697592, 0.017219550437597812, 0.04531795744923671, 0.00487914033626523, 0.0010832163857604387, 0.007237661962911473, 0.0037952153545474454, 0.03624552448186694, 8.56537211084601e-05, 0.0028624276012147003, 0.0016542142841297985, 0.0025711487789064737, 0.006174705142321824, 0.005545352106510327, 0.022213339709194774, 0.016792857637981072, 0.00893785926095322, 0.0726572688371982, 0.017913379712691378, 0.01704895982995209, 0.012045332214685579, 0.006174705142321824, 0.0020007412120077502, 0.00251121497502714, 0.013192628479645403, 0.051964313550828194, 0.006174705142321824, 0.02055892118744381, 0.0023240357383178, 0.008235041358247281, 0.007622891505037446, 0.0037952153545474454, 0.00481337003115552, 0.00031192927079150523, 0.05479203322186133, 0.025610590919955677, 0.0037952153545474454, 0.05479203322186133, 0.0028624276012147003, 0.009670897657175761, 0.0005617663833460786, 0.0007782037635891297, 0.05479203322186133, 0.0726572